<a href="https://colab.research.google.com/github/arunt-sjsu/DL_Project/blob/main/assignment_7_Catchup_1/FEBRL_Data_with_Active_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use Active Learning to Link FEBRL People Data
The following notebook is a replication of the active learning example using FEBRL dataset. 

We use dedeupe dataset to perform an active learning excercise and apply simsum classification.

<a href="https://colab.research.google.com/github/rachhouse/intro-to-data-linking/blob/main/tutorial_notebooks/03_Link_FEBRL_Data_with_Active_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

In [1]:
import requests

tutorial_functions_url = "https://raw.githubusercontent.com/rachhouse/intro-to-data-linking/main/tutorial_notebooks/linking_tutorial_functions.py"
r = requests.get(tutorial_functions_url)
    
with open("linking_tutorial_functions.py", "w") as fh:
  fh.write(r.text)
    
!pip install -q altair dedupe dedupe-variable-name jellyfish recordlinkage 
!pip install -U numpy


     |████████████████████████████████| 90 kB 2.6 MB/s 
     |████████████████████████████████| 764 kB 44.1 MB/s 
     |████████████████████████████████| 137 kB 69.6 MB/s 
     |████████████████████████████████| 944 kB 62.4 MB/s 
     |████████████████████████████████| 155 kB 70.2 MB/s 
     |████████████████████████████████| 71 kB 6.7 MB/s 
     |████████████████████████████████| 103 kB 52.1 MB/s 
     |████████████████████████████████| 79 kB 8.0 MB/s 
     |████████████████████████████████| 3.6 MB 86.0 MB/s 
     |████████████████████████████████| 46 kB 3.0 MB/s 
     |████████████████████████████████| 531 kB 72.5 MB/s 
     |████████████████████████████████| 251 kB 72.0 MB/s 
     |████████████████████████████████| 253 kB 85.8 MB/s 
     |████████████████████████████████| 787 kB 71.0 MB/s 
     |████████████████████████████████| 203 kB 67.3 MB/s 
     |████████████████████████████████| 888 kB 68.4 MB/s 
     |████████████████████████████████| 743 kB 72.5 MB/s 


In [1]:
import datetime
import itertools
import os
import pathlib
import re
from typing import Any, Dict, Optional

import dedupe
import pandas as pd

import linking_tutorial_functions as tutorial

INFO:root:Generating grammar tables from /usr/lib/python3.7/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.7/lib2to3/PatternGrammar.txt


## Define Working Filepaths

For convenience, we'll define a `pathlib.Path` to reference our current working directory.

In [2]:
WORKING_DIR = pathlib.Path(os.path.abspath(''))
WORKING_DIR

PosixPath('/content')

## Load Training Dataset and Ground Truth Labels

In [5]:
df_A, df_B, df_ground_truth = tutorial.load_febrl_training_data(True)

Let's take a quick look at our training dataset to refresh on the columns, formats, and data.

In [6]:
df_A.head()

,first_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,age,phone_number,soc_sec_id
person_id_A,,,,,,,,,,,,
fbc4143d-15f9-4f27-b5f0-dedbadce6616,matilda,struck,8,ballard place,,west perth,2470,qld,19611002,32,03 05903135,8276847
48a56cad-7ba6-45e1-97cd-517ba65bdab5,lachlan,eglinton,36,kambalda crescent,villa 427,auburn,5109,,19260108,27,,9937958
b1792d21-e4be-4b86-8dea-454ffa5194c5,mikayla,asher,588,britten-jones drive,,miami,4218,nsw,19251102,32,03 33770501,7017310
96653d73-bebc-4459-94f3-c3f0a8c514d4,grace,bristow,7,,wandella park snowy,cardiff,6163,nsw,19400120,,07 37864073,3535974
41f038b8-77c0-45a5-9e1f-e62b8637ffd1,wilson,bishop,11,chisholm street,,bronte,2490,nsw,19210305,27,04 15209769,5573522


## Data Augmentation

We'll do minimal data augmentation before feeding our training data to `dedupe`; we just want to format the date of birth data as `mm/dd/yy`, and ensure all columns are in string format and stripped of trailing/leading whitespace. Additionally, `dedupe` requires input data to be in dictionaries, using the record id as the key and the record metadata as the value. So, we'll convert our dataframes to this format.

In [7]:
def format_dob(dob: str) -> Optional[str]:
    """ Transform date of birth format from YYYYMMDD to mm/dd/yy.
        If DOB cannot be transformed, return None.
    """
    try:
        if re.match(r"\d{8}", dob):
            return (datetime.datetime.strptime(dob, "%Y%m%d")).strftime("%m/%d/%y")
    except:
        pass

    return None

def strip_and_null(x: Any) -> Optional[str]:
    """ Stringify incoming variable, remove trailing/leading whitespace
        and return resulting string. Return None if resulting string is empty.
    """
    x = str(x).strip()
    
    if x == "":
        return None
    else:
        return x
    
def convert_df_to_dict(df: pd.DataFrame) -> Dict[str, Dict]:
    """ Convert pandas DataFrame to dict keyed by record id.
        Convert all fields to strings or Nones to satisfy dedupe.
        Transform date format of date_of_birth field.
    """    

    for col in df.columns:
        df[col] = df[col].apply(lambda x: strip_and_null(x))

    df["date_of_birth"] = df["date_of_birth"].apply(lambda x: format_dob(x))    

    return df.to_dict("index")

In [8]:
records_A = convert_df_to_dict(df_A)
records_B = convert_df_to_dict(df_B)

We can examine a small sample of the resulting transformed records:

In [9]:
[records_A[k] for k in list(records_A.keys())[0:2]]

[{'address_1': 'ballard place',
  'address_2': None,
  'age': '32',
  'date_of_birth': '10/02/61',
  'first_name': 'matilda',
  'phone_number': '03 05903135',
  'postcode': '2470',
  'soc_sec_id': '8276847',
  'state': 'qld',
  'street_number': '8',
  'suburb': 'west perth',
  'surname': 'struck'},
 {'address_1': 'kambalda crescent',
  'address_2': 'villa 427',
  'age': '27',
  'date_of_birth': '01/08/26',
  'first_name': 'lachlan',
  'phone_number': None,
  'postcode': '5109',
  'soc_sec_id': '9937958',
  'state': None,
  'street_number': '36',
  'suburb': 'auburn',
  'surname': 'eglinton'}]

## Prepare Training

When we linked our data via SimSum and supervised learning, we defined our blockers and comparators manually with `recordlinkage`. The `dedupe` library takes an active learning approach to blocking and classification and will use our feedback gathered during the labeling session to learn blocking rules and train a classifier. 

To prepare our `dedupe.RecordLink` object for training, first we'll define the fields that we think `dedupe` should pay attention to when matching records - these definitions will serve as the comparators. The `field` contains the name of the attribute to use for comparison, and the `type` defines the comparison type.

In [22]:
%%time

fields = [
    { "field" : "first_name", "type" : "Name" },
    { "field" : "surname", "type" : "Name" },
    { "field" : "address_1", "type" : "ShortString" },
    { "field" : "address_2", "type" : "ShortString" },
    { "field" : "suburb", "type" : "ShortString" },
    { "field" : "postcode", "type" : "Exact" },
    { "field" : "state", "type" : "Exact" },
    { "field" : "date_of_birth", "type" : "DateTime" },
    { "field" : "soc_sec_id", "type" : "Exact" },
]

linker = dedupe.RecordLink(fields)
linker.prepare_training(records_A, records_B)

INFO:dedupe.canopy_index:Removing stop word re
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (dayPredicate, date_of_birth)


CPU times: user 45 s, sys: 1.36 ms, total: 45 s
Wall time: 44.7 s


## Active Learning Labeling Session!

At this point, we're ready to provide feedback to `dedupe` via an active learning labeling session. For this, `dedupe` supplies a convenience method to iterate through pairs it is uncertain about. As you provide feedback for each pair, dedupe learns blocking rules and recalculates its linking model weights.

You can use `y` (yes, match), `n` (no, not match), and `u` (unsure) to provide feedback on candidate links. When you're ready to exit the labeling session, use `f`.

In [23]:
dedupe.console_label(linker)

first_name : kristo
surname : millar
address_1 : ey place
address_2 : student village
suburb : manly vale
postcode : 2574
state : vic
date_of_birth : 06/05/08
soc_sec_id : 3943820

first_name : kristo
surname : millat
address_1 : None
address_2 : student village
suburb : manly vale
postcode : 2574
state : vic
date_of_birth : None
soc_sec_id : 3943820

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


y


first_name : kiandra
surname : dunstone
address_1 : None
address_2 : None
suburb : oaklands park
postcode : 6163
state : wa
date_of_birth : 10/29/11
soc_sec_id : 5277244

first_name : kiandra
surname : dunstone
address_1 : None
address_2 : None
suburb : oaklands park
postcode : 6163
state : wa
date_of_birth : None
soc_sec_id : 5277244

1/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


y


INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:SimplePredicate: (dayPredicate, date_of_birth)
INFO:dedupe.training:SimplePredicate: (firstTwoTokensPredicate, address_2)
first_name : hayden
surname : adrain
address_1 : None
address_2 : calma
suburb : concord west
postcode : 4362
state : wa
date_of_birth : 08/20/86
soc_sec_id : 9761425

first_name : hayden
surname : adrain
address_1 : None
address_2 : calma
suburb : concord west
postcode : 4362
state : wm
date_of_birth : None
soc_sec_id : 9761425

2/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

first_name : joshua
surname : beelitz
address_1 : None
address_2 : None
suburb : margaret river
postcode : 2204
state : vic
date_of_birth : 02/10/77
soc_sec_id : 2754936

first_name : joshua
surname : van meer
address_1 : barnard circuit
address_2 : None
suburb : yarraville
postcode : 2750
state : vic
date_of_birth : 12/01/79
soc_sec_id : 8908531

26/10 positive, 10/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


first_name : dylan
surname : paine
address_1 : macalister crescent
address_2 : westmead accom
suburb : None
postcode : 2148
state : sa
date_of_birth : None
soc_sec_id : 2677567

first_name : alana
surname : paine
address_1 : None
address_2 : None
suburb : nicholls
postcode : 3821
state : qld
date_of_birth : 04/21/13
soc_sec_id : 8636055

26/10 positive, 11/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling


We can now train our linker, based on the labeling session feedback.

In [24]:
%%time
linker.train()

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.000100, score 0.7109724930312558
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (wholeFieldPredicate, postcode), TfidfTextSearchPredicate: (0.2, first_name))
INFO:dedupe.training:(SimplePredicate: (threeDayPredicate, date_of_birth), PartialPredicate: (fingerprint, surname, Surname), SimplePredicate: (commonFourGram, first_name))
INFO:dedupe.training:(SimplePredicate: (commonTwoTokens, surname), PartialPredicate: (sameFiveCharStartPredicate, surname, Surname), LevenshteinSearchPredicate: (4, suburb))
INFO:dedupe.training:(SimplePredicate: (doubleMetaphone, suburb), PartialIndexLevenshteinSearchPredicate: (3, surname, Surname), SimplePredicate: (exclusiveMonthPredicate, date_of_birth))
INFO:dedupe.training:(SimplePredicate: (wholeFieldPredicate, date_of_birth), PartialIndexTfidfNGramSearchPredicate: (0.2, first_name, Surname), SimplePredicat

CPU times: user 12.4 s, sys: 516 ms, total: 12.9 s
Wall time: 12.3 s


Let's persist our training data (captured during in the labeling session), as well as the learned model weights.

In [25]:
ACTIVE_LEARNING_DIR = WORKING_DIR / "dedupe_active_learning"
ACTIVE_LEARNING_DIR.mkdir(parents=True, exist_ok=True)

SETTINGS_FILE = ACTIVE_LEARNING_DIR / "dedupe_learned_settings"
TRAINING_FILE = ACTIVE_LEARNING_DIR / "dedupe_training.json"

with open(TRAINING_FILE, "w") as fh:
    linker.write_training(fh)
    
with open(SETTINGS_FILE, "wb") as sf:
    linker.write_settings(sf)

## Examine Learned Blockers

Now, let's take a look at the predicates (blockers) that `dedupe` learned during our active learning labeling session. Note that `dedupe` can learn composite predicates/blockers, i.e. individual predicates can be combined with logical operators.

In [26]:
linker.predicates

((SimplePredicate: (wholeFieldPredicate, postcode),
  TfidfTextSearchPredicate: (0.2, first_name)),
 (SimplePredicate: (threeDayPredicate, date_of_birth),
  PartialPredicate: (fingerprint, surname, Surname),
  SimplePredicate: (commonFourGram, first_name)),
 (SimplePredicate: (commonTwoTokens, surname),
  PartialPredicate: (sameFiveCharStartPredicate, surname, Surname),
  LevenshteinSearchPredicate: (4, suburb)),
 (SimplePredicate: (doubleMetaphone, suburb),
  PartialIndexLevenshteinSearchPredicate: (3, surname, Surname),
  SimplePredicate: (exclusiveMonthPredicate, date_of_birth)),
 (SimplePredicate: (wholeFieldPredicate, date_of_birth),
  PartialIndexTfidfNGramSearchPredicate: (0.2, first_name, Surname),
  SimplePredicate: (wholeFieldPredicate, state)),
 (SimplePredicate: (fingerprint, address_1),
  PartialPredicate: (commonSixGram, surname, Surname),
  SimplePredicate: (commonSixGram, first_name)))

Next, let's examine the resulting candidate pairs and look at our blocking efficiency. The `.pairs` method will give us all candidate record pairs that are generated by blocking with the learned blockers.

In [15]:
candidate_pairs = [x for x in linker.pairs(records_A, records_B)]
print(f"{len(candidate_pairs):,} candidate pairs generated from blocking.")

0 candidate pairs generated from blocking.


You'll notice that, in contrast to `recordlinkage`, our post-blocking candidate pairs contain both the record ids as well as the record metadata.

In [27]:
candidate_pairs[0]

IndexError: ignored

We can assemble our candidate pair ids into an indexed pandas dataframe for easier comparision with our known true links.

In [ ]:
df_candidate_links = pd.DataFrame(
    [(x[0][0], x[1][0]) for x in candidate_pairs]
).rename(columns={0 : "person_id_A", 1 : "person_id_B"}).set_index(["person_id_A", "person_id_B"])

df_candidate_links.head()

Now, let's take a look at our learned blocker performance.

In [ ]:
max_candidate_pairs = df_A.shape[0]*df_B.shape[0]

print(f"{max_candidate_pairs:,} total possible pairs.")

# Calculate search space reduction.
search_space_reduction = round(1 - len(candidate_pairs)/max_candidate_pairs, 6)
print(f"\n{len(candidate_pairs):,} pairs after full blocking: {search_space_reduction}% search space reduction.")

# Calculate retained true links percentage.
total_true_links = df_ground_truth.shape[0]
true_links_after_blocking = pd.merge(
    df_ground_truth,
    df_candidate_links,
    left_index=True,
    right_index=True,
    how="inner"
).shape[0]

retained_true_link_percent = round((true_links_after_blocking/total_true_links) * 100, 2)
print(f"{retained_true_link_percent}% true links retained after blocking.")

## Score Pairs and Examine Learned Classifier

After `dedupe` has trained blockers and a classification model based on our labeling session, we can link the records in our training dataset via the `.join` method.

In [ ]:
%%time
linked_records = linker.join(records_A, records_B, threshold=0.0, constraint="one-to-one")

`linker.join` will return the links, along with a model confidence.

In [ ]:
linked_records[0:3]

We'll format the `dedupe` linker predictions into a format that we can use with our existing evaluation functions.

In [ ]:
df_predictions = pd.DataFrame(
    [ {"person_id_A" : x[0][0], "person_id_B" : x[0][1], "model_score" : x[1]} for x in linked_records]
)

df_predictions = df_predictions.set_index(["person_id_A", "person_id_B"])

df_predictions = pd.merge(
    df_predictions,
    df_ground_truth,
    left_index=True,
    right_index=True,
    how="left",
)

df_predictions["ground_truth"].fillna(False, inplace=True)
df_predictions

## Choosing a Linking Model Score Threshold

The `dedupe` `.join` method that we used to score our training data directly incorporates the learned blockers. Thus, note that the scored pairs appearing on the distribution represent blocked pairs, and that our blockers *significantly* reduced the candidate pair search space.

### Model Score Distribution

In [ ]:
df_predictions["ground_truth"].value_counts()

In [ ]:
tutorial.plot_model_score_distribution(df_predictions)

### Precision and Recall vs. Model Score

In [ ]:
df_eval = tutorial.evaluate_linking(
    df=df_predictions
)

In [ ]:
df_eval.head()

In [ ]:
tutorial.plot_precision_recall_vs_threshold(df_eval)

## Iterating with Active Learning

When using active learning, we iterate on our linking solution, and incorporate progressively more labeled training data. Perhaps we're not satisfied with the current performance of the blockers or classifier, and we'd like to create more labeled examples for dedupe to train on.

Recall that earlier, we saved off our existing training data from the first labeling session. We can load this persisted data into a `dedupe` linker, and kick off another labeling session. Perhaps, after investigating the data during our first cycle, we don't think that dedupe should include `address_1` and `address2` in its comparators.

### Tweak the Linker and Use Existing Training Data

In [ ]:
%%time

fields = [
    { "field" : "first_name", "type" : "Name" },
    { "field" : "surname", "type" : "Name" },
    { "field" : "suburb", "type" : "ShortString" },
    { "field" : "postcode", "type" : "Exact" },
    { "field" : "state", "type" : "Exact" },
    { "field" : "date_of_birth", "type" : "DateTime" },
    { "field" : "soc_sec_id", "type" : "Exact" },
]

linker2 = dedupe.RecordLink(fields)

with open(TRAINING_FILE, "r") as fh:
    linker2.prepare_training(records_A, records_B, training_file=fh)

Now, we can kick off a second active learning/labeling session.

In [ ]:
dedupe.console_label(linker2)

### Retrain the Linker and Examine Blocking Performance

Now, let's retrain, and examine blocker performance. Ideally, we see an improved true link retention following our second labeling session.

In [ ]:
%%time
linker2.train()

In [ ]:
candidate_pairs = [x for x in linker2.pairs(records_A, records_B)]
print(f"{len(candidate_pairs):,} candidate pairs generated from blocking.")

df_candidate_links = pd.DataFrame(
    [(x[0][0], x[1][0]) for x in candidate_pairs]
).rename(columns={0 : "person_id_A", 1 : "person_id_B"}).set_index(["person_id_A", "person_id_B"])

max_candidate_pairs = df_A.shape[0]*df_B.shape[0]

print(f"{max_candidate_pairs:,} total possible pairs.")

# Calculate search space reduction.
search_space_reduction = round(1 - len(candidate_pairs)/max_candidate_pairs, 6)
print(f"\n{len(candidate_pairs):,} pairs after full blocking: {search_space_reduction}% search space reduction.")

# Calculate retained true links percentage.
total_true_links = df_ground_truth.shape[0]
true_links_after_blocking = pd.merge(
    df_ground_truth,
    df_candidate_links,
    left_index=True,
    right_index=True,
    how="inner"
).shape[0]

retained_true_link_percent = round((true_links_after_blocking/total_true_links) * 100, 2)
print(f"{retained_true_link_percent}% true links retained after blocking.")

### Evaluate Classification Performance

In [ ]:
%%time
linked_records = linker2.join(records_A, records_B, threshold=0.0, constraint="one-to-one")

In [ ]:
df_predictions = pd.DataFrame(
    [ {"person_id_A" : x[0][0], "person_id_B" : x[0][1], "model_score" : x[1]} for x in linked_records]
)

df_predictions = df_predictions.set_index(["person_id_A", "person_id_B"])

df_predictions = pd.merge(
    df_predictions,
    df_ground_truth,
    left_index=True,
    right_index=True,
    how="left",
)

df_predictions["ground_truth"].fillna(False, inplace=True)
df_predictions

In [ ]:
df_predictions["ground_truth"].value_counts()

In [ ]:
tutorial.plot_model_score_distribution(df_predictions)

In [ ]:
df_eval = tutorial.evaluate_linking(
    df=df_predictions
)

tutorial.plot_precision_recall_vs_threshold(df_eval)